### 2


In [37]:
import numpy as np

In [38]:
# вспомогательные методы для вывода векторов и матриц
def print_r(matr):
    print(np.round(matr, 6))
    print()
    

In [50]:
def create_A(M, N, is_dy):
    A = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            A[i][j] = (i - j) ** 2
    if is_dy:
        dy = [0, 8, 1, 9, 9, 9]
        for i in range(min(6, M)):
            A[i][N - 1] = dy[i]
    return A


def gauss_solve(a, b):
    n = b.shape[0]
    x = np.empty(n)
    x[n - 1] =  b[n - 1] / a[n - 1][n - 1]
    diag = n - 2
    for i in range(n - 2, -1, -1):
        s = b[i]
        for j in range(n - 1, diag, -1):
            s -= a[i][j] * x[j]
        x[i] = s / a[i][i]
        diag -= 1
    return x


def solve(a, b, r):
    y = gauss_solve(a[:r, :r], b[:r])
    x = np.zeros(a.shape[1])
    for i in range(r):
        x[i] = y[i]
    return x


def get_max_norm_col_index(A):
    M, N = A.shape
    norms = np.empty(N)
    index = 0
    max_norm = np.linalg.norm(np.array([A[j, 0] for j in range(M)]))
    norms[0] = max_norm
    for i in range(1, N):
        norm = np.linalg.norm(np.array([A[j, i] for j in range(M)]))
        norms[i] = norm
        if (norm > max_norm):
            max_norm = norm
            index = i
    return index, max_norm, norms


def get_householder_vector(x):   
    w = x.copy()
    w[0] = x[0] + np.copysign(np.linalg.norm(x), x[0])
    beta = 2 / (w.T @ w)
    return w, beta


def get_permutation_matrix(p):
    n = len(p)
    PI = np.zeros((n, n))
    for i in range(n):
        PI[i, p[i]] = 1
    return PI.T


def householder_transform(A, b, to_log, get_Q=False):
    m, n = A.shape
    R = A.copy()
    b_ = b.copy()
    p = np.arange(n)
    Q_T = np.identity(m)
    rank = 0
    for j in range(n):
        j_max_norm, max_norm, norms = get_max_norm_col_index(R[j:,  j:])
        if (to_log):
            print('Матрица до зануления столбца:')
            print_r(R)
            print(f'Нормы столцов подматрицы [{j}, {n}] x [{j}, {n}]:')
            print_r(norms)
            print(f'макс. норма подматрицы в столбце № {j_max_norm} подматрицы\nв столбце № {j + j_max_norm} всей матрицы\n')
        if (abs(max_norm) < 1e-10):
            break
        if (j_max_norm != 0):
            j1 = j + j_max_norm
            R[:, [j, j1]] = R[:, [j1, j]] 
            t = p[j]
            p[j] = p[j1]
            p[j1] = t
            if (to_log):
                print(f'Меняем столбцы {j} и {j1}, получаем матрицу с переставленными столбцами:')
                print_r(R)
        elif (to_log):
            print('Менять столбцы не нужно')
        w, beta = get_householder_vector(R[j:, j, np.newaxis])
        R[j:,  j:] -= beta * w @ (R[j:,  j:].T @ w).T
        b_[j:] -= beta * w @ (b_[j:].T @ w).T
        if (get_Q):
            H = np.identity(m)
            H[j:, j:] -= beta * (w @ w.T)
            Q_T = H @ Q_T
        rank = j + 1
        if (to_log):
            print(f'Матрица с зануленным столбцом {j}:')
            print_r(R)
            print('Преобразованная правая часть:')
            print_r(b_)
            print('--------------------------------------------------')
    return Q_T.T, R, b_, get_permutation_matrix(p), rank


def get_ro(R, x, b, r):
    R11 = R[:r, :r]
    R12 = R[:r, r:]
    y = x[:r]
    z = x[r:]
    c = b[:r]
    d = b[r:]
    return np.linalg.norm(R11 @ y + R12 @ z - c) ** 2 + np.linalg.norm(d) ** 2

In [51]:
N = 6
M = N + 2
AA = np.random.rand(M, N)
xx = np.array([i for i in range(N)])
bb = AA @ xx
print_r(AA)
print_r(bb)
#print_r(np.linalg.solve(AA, bb))

Q, R, b_, PI, rank = householder_transform(AA, bb, True, True)
x_ = solve(R, b_, rank)
ro = get_ro(R, x_, b_, rank)
print('Матрица R')
print_r(R)
print('Преобразованная правая часть')
print_r(b_)
print('Полученное решение x')
print_r(PI @ x_)
print('Норма невязки')
print_r(ro)
print('--')

[[0.681098 0.613541 0.711186 0.009471 0.884938 0.5943  ]
 [0.53936  0.578949 0.110495 0.673421 0.301186 0.175643]
 [0.153793 0.22138  0.037745 0.075813 0.791539 0.733877]
 [0.725287 0.075111 0.343977 0.49953  0.568696 0.420419]
 [0.207506 0.125499 0.97608  0.047017 0.51796  0.789104]
 [0.355858 0.434293 0.413568 0.990457 0.468409 0.883762]
 [0.721349 0.441297 0.066777 0.137559 0.645772 0.576675]
 [0.913061 0.469082 0.056058 0.774794 0.982149 0.240783]]

[ 8.575576  4.903159  7.359846  6.638538  8.236069 10.525246  6.453993
  8.038094]

Матрица до зануления столбца:
[[0.681098 0.613541 0.711186 0.009471 0.884938 0.5943  ]
 [0.53936  0.578949 0.110495 0.673421 0.301186 0.175643]
 [0.153793 0.22138  0.037745 0.075813 0.791539 0.733877]
 [0.725287 0.075111 0.343977 0.49953  0.568696 0.420419]
 [0.207506 0.125499 0.97608  0.047017 0.51796  0.789104]
 [0.355858 0.434293 0.413568 0.990457 0.468409 0.883762]
 [0.721349 0.441297 0.066777 0.137559 0.645772 0.576675]
 [0.913061 0.469082 0.056058 

#### 0. Тестовые пример с точным решением и пример не имеющего точного решения

In [60]:
N = 6
M = N + 2
A = create_A(M, N, True)
x = np.array([1.0 for i in range(N)])
b = A @ x
b_f = b.copy()
b_f[0] = 0

print('Матрица A')
print_r(A)
print('Вектор b для задачи с точным решением')
print_r(b)
print('Вектор b для для задачи не имеющего точного решения (rank(A|b) > rank(A))')
print_r(b_f)
print('Решение x')
print_r(x)

Матрица A
[[ 0.  1.  4.  9. 16.  0.]
 [ 1.  0.  1.  4.  9.  8.]
 [ 4.  1.  0.  1.  4.  1.]
 [ 9.  4.  1.  0.  1.  9.]
 [16.  9.  4.  1.  0.  9.]
 [25. 16.  9.  4.  1.  9.]
 [36. 25. 16.  9.  4.  1.]
 [49. 36. 25. 16.  9.  4.]]

Вектор b для задачи с точным решением
[ 30.  23.  11.  24.  39.  64.  91. 139.]

Вектор b для для задачи не имеющего точного решения (rank(A|b) > rank(A))
[  0.  23.  11.  24.  39.  64.  91. 139.]

Решение x
[1. 1. 1. 1. 1. 1.]



#### 1, 2, 3. Решение СЛАУ в смысле НК

In [61]:
_, R, b_, PI, rank = householder_transform(A, b, False)
x_ = solve(R, b_, rank)
ro = get_ro(R, x_, b_, rank)
print('Матрица R')
print_r(R)
print('Преобразованная правая часть')
print_r(b_)
print(f'Ранг матрицы = {rank}')
print('Полученное решение x')
print_r(PI @ x_)
print('Норма невязки')
print_r(ro)
print_r(A@x - b)

Матрица R
[[-68.381284  -9.417782 -10.148976 -30.710157 -47.498377 -18.016626]
 [  0.       -19.060572  -2.015635  -4.342915  -0.874615 -10.404901]
 [  0.         0.        14.762639  -4.770853  -3.57814   -3.57814 ]
 [  0.         0.         0.         3.35623    2.517172   2.517172]
 [  0.        -0.         0.         0.        -0.        -0.      ]
 [  0.         0.         0.        -0.        -0.        -0.      ]
 [  0.         0.        -0.        -0.         0.        -0.      ]
 [  0.        -0.         0.         0.        -0.         0.      ]]

Преобразованная правая часть
[-184.173202  -36.698637    2.835506    8.390574    0.         -0.
   -0.          0.      ]

Ранг матрицы = 4
Полученное решение x
[1.25 0.   2.5  0.   1.25 1.  ]

Норма невязки
0.0

[0. 0. 0. 0. 0. 0. 0. 0.]



In [62]:
_, R, b_, PI, rank = householder_transform(A, b_f, True)
x_ = solve(R, b_, rank)
ro = get_ro(R, x_, b_, rank)
print('rank(A|b) > rank(A)')
print('Матрица R')
print_r(R)
print('Преобразованная правая часть')
print_r(b_)
print('Полученное решение x')
print_r(PI @ x_)
print('Норма невязки')
print_r(ro)

Матрица до зануления столбца:
[[ 0.  1.  4.  9. 16.  0.]
 [ 1.  0.  1.  4.  9.  8.]
 [ 4.  1.  0.  1.  4.  1.]
 [ 9.  4.  1.  0.  1.  9.]
 [16.  9.  4.  1.  0.  9.]
 [25. 16.  9.  4.  1.  9.]
 [36. 25. 16.  9.  4.  1.]
 [49. 36. 25. 16.  9.  4.]]

Нормы столцов подматрицы [0, 6] x [0, 6]:
[68.381284 47.707442 31.559468 21.260292 21.260292 18.027756]

макс. норма подматрицы в столбце № 0 подматрицы
в столбце № 0 всей матрицы

Менять столбцы не нужно
Матрица с зануленным столбцом 0:
[[-68.381284 -47.498377 -30.710157 -18.016626  -9.417782 -10.148976]
 [  0.        -0.709235   0.492403   3.604912   8.628293   7.851583]
 [  0.        -1.836939  -2.030389  -0.580352   2.513173   0.40633 ]
 [  0.        -2.383112  -3.568376  -3.555792  -2.34536    7.664243]
 [  0.        -2.347755  -4.121557  -5.321408  -5.947307   6.625321]
 [  0.        -1.730867  -3.689933  -5.8772    -8.292668   5.289564]
 [  0.        -0.532448  -2.273504  -5.223169  -9.381441  -4.343028]
 [  0.         1.247502   0.127

#### 4. Вычисление в явном виде матрицы Q

In [21]:
N = 5
M = N + 2
A = create_A(M, N, True)
x = np.array([1 for i in range(N)])
b = A @ x

print('Матрица A')
print_r(A)
print('Вектор b для задачи с точным решением')
print_r(b)

Матрица A
[[ 0.  1.  4.  9.  0.]
 [ 1.  0.  1.  4.  8.]
 [ 4.  1.  0.  1.  1.]
 [ 9.  4.  1.  0.  9.]
 [16.  9.  4.  1.  9.]
 [25. 16.  9.  4.  9.]
 [36. 25. 16.  9.  4.]]

Вектор b для задачи с точным решением
[14. 14.  7. 23. 39. 63. 90.]



In [22]:
Q, R, b_, PI, rank = householder_transform(A, b, False, True)

print('Матрица Q')
print_r(Q)
print('Матрица R')
print_r(R)

Матрица Q
[[ 0.       -0.        0.86762   0.127974  0.157811 -0.048349 -0.451238]
 [-0.020966 -0.606531  0.391092 -0.21308  -0.119678  0.109811  0.637921]
 [-0.083863  0.005137  0.02026   0.929756 -0.112141  0.243248  0.237356]
 [-0.188691 -0.51783  -0.149966  0.172467 -0.442923 -0.580992 -0.332086]
 [-0.335451 -0.371592 -0.192349  0.109464  0.831226 -0.089392 -0.038513]
 [-0.524142 -0.183573 -0.081582 -0.161657 -0.233597  0.689805 -0.358314]
 [-0.754765  0.438368  0.166519 -0.076893 -0.080699 -0.324131  0.304875]]

Матрица R
[[-47.69696  -12.705212  -9.392632 -18.344985 -31.113094]
 [  0.       -12.750591   0.418442   2.751004   2.611523]
 [  0.         0.        10.373204   4.872282   1.414547]
 [  0.         0.        -0.        -1.77606   -1.77606 ]
 [  0.         0.         0.        -0.        -0.      ]
 [  0.         0.         0.        -0.         0.      ]
 [  0.         0.        -0.         0.         0.      ]]



#### 5. Формирование матрицы перестановок. Проверка равенства AП = QR.

In [23]:
N = 4
M = N + 2
A = create_A(M, N, True)
x = np.array([1 for i in range(N)])
b = A @ x

print('Матрица A')
print_r(A)
print('Вектор b для задачи с точным решением')
print_r(b)

Матрица A
[[ 0.  1.  4.  0.]
 [ 1.  0.  1.  8.]
 [ 4.  1.  0.  1.]
 [ 9.  4.  1.  9.]
 [16.  9.  4.  9.]
 [25. 16.  9.  9.]]

Вектор b для задачи с точным решением
[ 5. 10.  6. 23. 38. 59.]



In [24]:
Q, R, b_, PI, rank = householder_transform(A, b, False, True)

print('Матрица Q')
print_r(Q)
print('Матрица R')
print_r(R)
print('Матрица Перестановок')
print_r(PI)
print('AП: ')
print_r(A @ PI)
print('QR: ')
print_r(Q @ R)
print('AП - QR: ')
print_r(A @ PI - Q @ R)

Матрица Q
[[ 0.       -0.        0.831875  0.407131  0.026807 -0.376179]
 [-0.03196  -0.793629  0.268012 -0.053686  0.062316  0.539013]
 [-0.127841  0.093577 -0.268635  0.880162 -0.037947  0.355824]
 [-0.287641 -0.501053 -0.285671  0.119744 -0.528124 -0.539765]
 [-0.511362 -0.152803 -0.160599  0.033242  0.786791 -0.263102]
 [-0.799003  0.294946  0.237886 -0.203061 -0.309843  0.284208]]

Матрица R
[[-31.288976 -14.765584  -9.556081 -18.664721]
 [  0.        -9.485649   0.748618   1.433271]
 [  0.         0.         4.808418   1.781342]
 [  0.        -0.        -0.        -1.183532]
 [  0.        -0.        -0.         0.      ]
 [  0.         0.        -0.         0.      ]]

Матрица Перестановок
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]

AП: 
[[ 0.  0.  4.  1.]
 [ 1.  8.  1.  0.]
 [ 4.  1.  0.  1.]
 [ 9.  9.  1.  4.]
 [16.  9.  4.  9.]
 [25.  9.  9. 16.]]

QR: 
[[-0. -0.  4.  1.]
 [ 1.  8.  1. -0.]
 [ 4.  1. -0.  1.]
 [ 9.  9.  1.  4.]
 [16.  9.  4.  9.]
 [25.  9.  9

#### 6. Решение слау с помощью SVD разложения

In [64]:
def get_svd_dec(A, to_log=False):
    m, n = A.shape
    U, S, VT = np.linalg.svd(A)
    if (to_log):
        print("U = ")
        print_r(U)
        print("V = ")
        print_r(VT.T)
        print('Проверки ортогональности')
        print('U_T * U')
        print_r(U.T @ U)
        print('V_T * V')
        print_r(VT.T @ VT)
        print("U_T A V")
        print_r(U.T @ A @ VT.T)
        print('S (синг. числа с диагонали)')
        print_r(S)
    return U, S, VT


def solve_by_svd(A, U, S, V, b, to_log=False):
    n = A.shape[1]
    rank = np.linalg.matrix_rank(A)
    c = U.T @ b
    y = np.zeros(n)
    for i in range(rank):
        y[i] = c[i] / S[i]
    x = V.T @ y
    S = U.T @ A @ V.T
    if (to_log):
        print('Полученное решение х')
        print_r(x_)
    print('Норма невзяки')
    print(np.linalg.norm(S @ y - U.T @ b) ** 2)
    return x


def solve_by_svd_pseudo_inv(A, U, S, V, b, to_log=False):
    m, n = A.shape
    S_pl = np.zeros((n, m))
    for i in range(n):
        S_pl[i][i] = 1 / S[i]
    A_pl = V.T @ S_pl @ U.T
    if (to_log):
        print('A+ A')
        print_r(A_pl @ A)
        print('A A+')
        print_r(A @ A_pl)
        print('___')
        print_r(np.linalg.inv(A.T @ A) @ A.T)
        print_r(A_pl)
    x = A_pl @ b
    if (to_log):
        print('Полученное решение х')
        print_r(x)
    return x

In [66]:
N = 6
M = N + 2
A = create_A(M, N, True)
x = np.array([1 for i in range(N)])
b = A @ x
b_f = b.copy()
b_f[0] = 0

print('Матрица A')
print_r(A)
print('Вектор b для задачи с точным решением')
print_r(b)
print_r(x)

Матрица A
[[ 0.  1.  4.  9. 16.  0.]
 [ 1.  0.  1.  4.  9.  8.]
 [ 4.  1.  0.  1.  4.  1.]
 [ 9.  4.  1.  0.  1.  9.]
 [16.  9.  4.  1.  0.  9.]
 [25. 16.  9.  4.  1.  9.]
 [36. 25. 16.  9.  4.  1.]
 [49. 36. 25. 16.  9.  4.]]

Вектор b для задачи с точным решением
[ 30.  23.  11.  24.  39.  64.  91. 139.]

[1 1 1 1 1 1]



In [67]:
U, S, VT = get_svd_dec(A, to_log=True)
x_ = solve_by_svd(A.copy(), U.copy(), S.copy(), VT.copy(), b_f.copy(), True)
print_r(A@x- b)

U = 
[[-0.060831 -0.838862  0.032459  0.112335  0.475229 -0.112985 -0.162256
  -0.11833 ]
 [-0.041447 -0.434937 -0.484021 -0.250436 -0.602438 -0.159451  0.351476
  -0.015044]
 [-0.046386 -0.13011  -0.095166  0.770626 -0.385612  0.215821 -0.298374
   0.306097]
 [-0.110581  0.073988 -0.532883  0.097618  0.452093  0.497925  0.425129
   0.236119]
 [-0.20725   0.178607 -0.469376  0.090266  0.004579  0.075346 -0.375203
  -0.741761]
 [-0.345708  0.183313 -0.366488 -0.072205  0.186581 -0.607341 -0.30618
   0.458186]
 [-0.516641  0.088539  0.237624  0.43098   0.020811 -0.334559  0.556262
  -0.247476]
 [-0.742171 -0.106745  0.246085 -0.349518 -0.151244  0.425244 -0.190854
   0.12221 ]]

V = 
[[-0.741479  0.238666 -0.10649   0.517322  0.145532  0.305133]
 [-0.517706  0.104575  0.155831 -0.261132 -0.125619 -0.78281 ]
 [-0.339001 -0.121677  0.240884 -0.545859 -0.496331  0.51763 ]
 [-0.205365 -0.440088  0.148669 -0.33686   0.787394  0.092636]
 [-0.116796 -0.850658 -0.120816  0.365865 -0.310975 -0.13

#### 7. Решение СЛАУ С помощью псевдообратной матрицы

In [28]:
N = 4
M = N + 2
A = create_A(M, N, True)
x = np.array([1.0 for i in range(N)])
b = A @ x

print('Матрица A')
print_r(A)
print('Вектор b для задачи с точным решением')
print_r(b)
print_r(x)

Матрица A
[[ 0.  1.  4.  0.]
 [ 1.  0.  1.  8.]
 [ 4.  1.  0.  1.]
 [ 9.  4.  1.  9.]
 [16.  9.  4.  9.]
 [25. 16.  9.  9.]]

Вектор b для задачи с точным решением
[ 5. 10.  6. 23. 38. 59.]

[1. 1. 1. 1.]



In [31]:
U, S, VT = get_svd_dec(A, to_log=True)
x_ = solve_by_svd_pseudo_inv(A.copy(), U.copy(), S.copy(), VT.copy(), b.copy(), True)
print('Полученное решение х')
print_r(x_)


U = 
[[-0.03484   0.129792  0.807861  0.432518 -0.005965 -0.377085]
 [-0.099346 -0.728685  0.399639 -0.070845  0.108893  0.531565]
 [-0.096275  0.018474 -0.309011  0.8757   -0.006901  0.357776]
 [-0.305153 -0.530031 -0.214493  0.098382 -0.573006 -0.491859]
 [-0.511678 -0.149871 -0.162899  0.030426  0.760968 -0.330439]
 [-0.790389  0.385275  0.140066 -0.174507 -0.283989  0.310043]]

V = 
[[-0.767981  0.193372 -0.413899  0.448887]
 [-0.458242  0.303996  0.091964 -0.830146]
 [-0.239037  0.227534  0.890259  0.313893]
 [-0.378259 -0.904667  0.166342 -0.104058]]

Проверки ортогональности
U_T * U
[[ 1. -0. -0. -0. -0.  0.]
 [-0.  1. -0. -0.  0. -0.]
 [-0. -0.  1. -0. -0. -0.]
 [-0. -0. -0.  1. -0. -0.]
 [-0.  0. -0. -0.  1. -0.]
 [ 0. -0. -0. -0. -0.  1.]]

V_T * V
[[ 1.  0.  0.  0.]
 [ 0.  1. -0. -0.]
 [ 0. -0.  1. -0.]
 [ 0. -0. -0.  1.]]

U_T A V
[[40.596556  0.        0.       -0.      ]
 [-0.        9.35443  -0.        0.      ]
 [-0.       -0.        4.521817  0.      ]
 [-0.       -0. 

#### 8.  Для больших систем


In [68]:
import time
Ns = [100, 500, 1000, 5000, 10000]

#### Хаусхолдер

In [69]:
prev_runtime = 0
for i in range(len(Ns)) :
    N = Ns[i]
    M = N + 2
    print(f'Столбцов {N}')
    A = create_A(M, N, False)
    x = np.array([1 for i in range(N)])
    b = A @ x
    start = time.time() * 1000.0
    _, R, b_, PI, rank = householder_transform(A, b, False)
    x_ = solve(R, b_, rank)
    runtime = time.time() * 1000.0 - start
    ro = get_ro(R, x_, b_, rank)
    print('Норма невязки')
    print(ro)
    print(f'Время вычисления в мс = {round(runtime)}')
    if (i > 0):
        print(f'Порядок матрицы увеличился в {N / Ns[i - 1]} раз, время выполнения увеличилось в {runtime / prev_runtime} раз')
    prev_runtime = runtime
    print('------------------------------------------------------------------------------')
    

Столбцов 100
Норма невязки
4.0657904585633195e-20
Время вычисления в мс = 24
------------------------------------------------------------------------------
Столбцов 500
Норма невязки
1.2473273443077298e-14
Время вычисления в мс = 11097
Порядок матрицы увеличился в 5.0 раз, время выполнения увеличилось в 454.1734794816199 раз
------------------------------------------------------------------------------
Столбцов 1000


KeyboardInterrupt: 

#### SVD

In [15]:
prev_runtime = 0
for i in range(len(Ns)) :
    N = Ns[i]
    M = N + 2
    print(f'Столбцов {N}')
    A = create_A(M, N, False)
    x = np.array([1 for i in range(N)])
    b = A @ x
    start = time.time() * 1000.0
    U, S, V = get_svd_dec(A, to_log=False)
    x_ = solve_by_svd(A.copy(), U.copy(), S.copy(), V.copy(), b.copy(), False)
    runtime = time.time() * 1000.0 - start
    print(f'Время вычисления в мс = {round(runtime)}')
    if (i > 0):
        print(f'Порядок матрицы увеличился в {N / Ns[i - 1]} раз, время выполнения увеличилось в {runtime / prev_runtime} раз')
    prev_runtime = runtime
    print('------------------------------------------------------------------------------')
    

Столбцов 100
Норма невзяки
5.376942081011013e-19
Время вычисления в мс = 16
------------------------------------------------------------------------------
Столбцов 500
Норма невзяки
1.3715409102379583e-13
Время вычисления в мс = 391
Порядок матрицы увеличился в 5.0 раз, время выполнения увеличилось в 24.168081058226626 раз
------------------------------------------------------------------------------
Столбцов 1000
Норма невзяки
2.4224416296193235e-11
Время вычисления в мс = 2400
Порядок матрицы увеличился в 2.0 раз, время выполнения увеличилось в 6.142367387334138 раз
------------------------------------------------------------------------------
Столбцов 5000
Норма невзяки
1.4452009800704267e-06
Время вычисления в мс = 159007
Порядок матрицы увеличился в 5.0 раз, время выполнения увеличилось в 66.24971442091478 раз
------------------------------------------------------------------------------
Столбцов 10000


KeyboardInterrupt: 

Изначальная матрица


NameError: name 'pr_matr' is not defined